- **접근방식  **
>**학습 데이터를 한달뒤를 예측하는 모형으로 학습.**  
>**예측한 4월과 실제 4월과의 차이를 비교하며 학습한다.**

- 예시


>- 학습데이터(xtrain)) : 2019.01 - 2020.02
>- 학습데이터(ytrain) : 2020.03
>- 예측데이터(predict) : 2019.02 - 2020.03 예측 > 2020.04
>- 비교데이터(Target) : 2020.04  




# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{}'.format
Dacon = pd.read_csv('data/date.csv')
Dacon_April = pd.read_csv('data/202004.csv')
Dacon

FileNotFoundError: [Errno 2] File data/date.csv does not exist: 'data/date.csv'

## 결측값 확인 및 변환

- **결측값 확인을 하였더니 세종 특별 자치시이기 떄문에 시군구가 없다.  **
- **NULL => 세종으로 대체**

In [3]:
# 결측값 '세종' 대체
Dacon['CARD_CCG_NM'] = Dacon['CARD_CCG_NM'].fillna('세종')
Dacon['HOM_CCG_NM'] = Dacon['HOM_CCG_NM'].fillna('세종')
Dacon_April['CARD_CCG_NM'] = Dacon_April['CARD_CCG_NM'].fillna('세종')
Dacon_April['HOM_CCG_NM'] = Dacon_April['HOM_CCG_NM'].fillna('세종')
print(Dacon.info(null_counts = True))
print(Dacon_April.info(null_counts = True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 12 columns):
REG_YYMM        24697792 non-null int64
CARD_SIDO_NM    24697792 non-null object
CARD_CCG_NM     24697792 non-null object
STD_CLSS_NM     24697792 non-null object
HOM_SIDO_NM     24697792 non-null object
HOM_CCG_NM      24697792 non-null object
AGE             24697792 non-null object
SEX_CTGO_CD     24697792 non-null int64
FLC             24697792 non-null int64
CSTMR_CNT       24697792 non-null int64
AMT             24697792 non-null int64
CNT             24697792 non-null int64
dtypes: int64(6), object(6)
memory usage: 2.2+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350322 entries, 0 to 1350321
Data columns (total 12 columns):
REG_YYMM        1350322 non-null int64
CARD_SIDO_NM    1350322 non-null object
CARD_CCG_NM     1350322 non-null object
STD_CLSS_NM     1350322 non-null object
HOM_SIDO_NM     1350322 non-null object
HOM_CCG_NM      1350322 non-null

In [31]:
corr = Dacon.corr()
corr_trsc = corr.AMT.sort_values(ascending=False).to_frame()
corr_trsc.style.background_gradient(cmap='magma')

,AMT
AMT,1
CSTMR_CNT,0.866339
CNT,0.778102
FLC,0.041099
REG_YYMM,0.00316692
SEX_CTGO_CD,-0.00542405


In [ ]:
DashBoard = Dacon.groupby(['REG_YYMM', 'STD_CLSS_NM', 'CARD_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'])['CSTMR_CNT', 'AMT', 'CNT'].sum()
DashBoard = DashBoard.reset_index()
DashBoard.to_excel('DashBoard.xlsx', index = False)

In [54]:
DashBoard

,REG_YYMM,STD_CLSS_NM,CARD_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,건강보조식품 소매업,강원,20s,1,1,4,311200,4
1,201901,건강보조식품 소매업,강원,30s,1,2,44,5169295,45
2,201901,건강보조식품 소매업,강원,30s,2,2,89,8618550,91
3,201901,건강보조식품 소매업,강원,40s,1,2,3,454420,3
4,201901,건강보조식품 소매업,강원,40s,1,3,53,6441165,60
...,...,...,...,...,...,...,...,...,...
197596,202003,휴양콘도 운영업,충북,40s,1,4,9,385700,10
197597,202003,휴양콘도 운영업,충북,50s,1,4,19,2084350,31
197598,202003,휴양콘도 운영업,충북,50s,2,4,4,99000,6
197599,202003,휴양콘도 운영업,충북,60s,1,5,6,228800,6


In [4]:
def 학습데이터(df, column):
    
    import sys
    mod = sys.modules[__name__]

    for state in df.CARD_SIDO_NM.unique():
        
        지역업종 = pd.DataFrame(sorted(df['STD_CLSS_NM'].unique()),columns=['STD_CLSS_NM']) # 업종기준으로 빈 데이터프레임 생성
        state_data = df[df.CARD_SIDO_NM == state]
        시 = pd.DataFrame(list(state_data.CARD_SIDO_NM[0:41]),columns=['CARD_SIDO_NM'])  # 시 빈 데이터프레임생성      
        
        for month in df.REG_YYMM.unique():
            month_data = state_data[state_data.REG_YYMM == month]
            month_data = month_data.groupby('STD_CLSS_NM')[column].sum().reset_index()
            month_data = month_data.rename(columns={column : month})
            지역업종 = pd.merge(지역업종, month_data, how = 'left', on = ['STD_CLSS_NM']).fillna(0)
            월별지역업종 = 시.join(지역업종)
    
        setattr(mod, '{}'.format(state), 월별지역업종) #state(시)의 이름으로 월별업종 데이터프레임을 각각 변수지정
        print('{} 데이터 프레임 저장 완료'.format(state))

    print('학습 데이터 생성')
    xtrain = pd.concat([강원, 경기, 경남, 경북, 광주, 대구, 대전, 부산, 서울, 세종, 울산, 인천, 전남, 전북, 제주, 충남, 충북])
    print('-- COMPLETE --')
    return xtrain


def April_Target(df, column):
    
    import sys
    mod = sys.modules[__name__]

    for state in df.CARD_SIDO_NM.unique():
        
        지역업종 = pd.DataFrame(sorted(df['STD_CLSS_NM'].unique()),columns=['STD_CLSS_NM']) # 업종기준으로 빈 데이터프레임 생성
        state_data = df[df.CARD_SIDO_NM == state]
        시 = pd.DataFrame(list(state_data.CARD_SIDO_NM[0:41]),columns=['CARD_SIDO_NM'])  # 시 빈 데이터프레임생성      
        
        month_data = state_data.groupby('STD_CLSS_NM')[column].sum().reset_index()
        month_data = month_data.rename(columns={column : 202004})
        지역업종 = pd.merge(지역업종, month_data, how = 'left', on = ['STD_CLSS_NM']).fillna(0)
        월별지역업종 = 시.join(지역업종)
    
        setattr(mod, '{}'.format(state), 월별지역업종) #state(시)의 이름으로 월별업종 데이터프레임을 각각 변수지정
        print('{} 데이터 프레임 저장 완료'.format(state))

    print('타겟 데이터 생성')
    Target = pd.concat([강원, 경기, 경남, 경북, 광주, 대구, 대전, 부산, 서울, 세종, 울산, 인천, 전남, 전북, 제주, 충남, 충북])
    print('-- COMPLETE --')
    return Target

In [16]:
# 월 업종별 금액 표
xtrain = 학습데이터(Dacon, 'CNT')
# 4월 업종별 금액 표
April = April_Target(Dacon_April, 'CNT')

강원 데이터 프레임 저장 완료
경기 데이터 프레임 저장 완료
경남 데이터 프레임 저장 완료
경북 데이터 프레임 저장 완료
광주 데이터 프레임 저장 완료
대구 데이터 프레임 저장 완료
대전 데이터 프레임 저장 완료
부산 데이터 프레임 저장 완료
서울 데이터 프레임 저장 완료
세종 데이터 프레임 저장 완료
울산 데이터 프레임 저장 완료
인천 데이터 프레임 저장 완료
전남 데이터 프레임 저장 완료
전북 데이터 프레임 저장 완료
제주 데이터 프레임 저장 완료
충남 데이터 프레임 저장 완료
충북 데이터 프레임 저장 완료
학습 데이터 생성
-- COMPLETE --
강원 데이터 프레임 저장 완료
경기 데이터 프레임 저장 완료
경남 데이터 프레임 저장 완료
경북 데이터 프레임 저장 완료
광주 데이터 프레임 저장 완료
대구 데이터 프레임 저장 완료
대전 데이터 프레임 저장 완료
부산 데이터 프레임 저장 완료
서울 데이터 프레임 저장 완료
세종 데이터 프레임 저장 완료
울산 데이터 프레임 저장 완료
인천 데이터 프레임 저장 완료
전남 데이터 프레임 저장 완료
전북 데이터 프레임 저장 완료
제주 데이터 프레임 저장 완료
충남 데이터 프레임 저장 완료
충북 데이터 프레임 저장 완료
타겟 데이터 생성
-- COMPLETE --


In [17]:
# 복사본 생성
xtrain_data = xtrain.copy()
April_data = April.copy()

In [6]:
# # 오브젝트 타입 컬럼 수치형으로 변환
object_columns = ['CARD_SIDO_NM', 'STD_CLSS_NM']
xtrain[object_columns] = xtrain[object_columns].apply(lambda x: x.astype('category').cat.codes)

# 월별금액 로그10으로 수치변환
for column in xtrain.columns[2:]:

    xtrain[column] = np.log10(xtrain[column] + 1)

xtrain

,CARD_SIDO_NM,STD_CLSS_NM,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003
0,0,0,8.171301447268446,8.166477099181714,8.039172219201445,8.0275472340772,8.170601209754631,7.991049175457223,7.966168644077878,7.995736632186553,8.312502339792603,8.064149687985251,7.991601011286078,7.9787843371519385,8.267192313473593,7.883693009769309,7.982538119978814
1,0,1,9.089622662171841,9.061543314417174,9.440987301527647,9.59823150234806,9.739883269767958,9.753105595129005,9.701818681398613,9.76343421508634,9.61893227533515,9.752209317132246,9.603952640240845,9.09341783323329,9.093807479667861,8.986080098628642,9.464757433136556
2,0,2,9.109571496563857,9.047767651770709,8.972771008503663,8.917864224621926,9.009226715649145,9.000705615354141,8.97925096599871,9.141432344201716,9.118406707251077,9.119874948471198,9.022280538664184,9.009198824348939,9.197709553758862,8.938126461135516,8.99774317368579
3,0,3,7.486526409789988,7.360236403265905,7.39834269086204,7.252989145197182,7.608337095779087,7.488517452217375,7.465439380417417,7.766392775401892,7.445345308403097,7.725407027037034,7.490521305965574,7.475920959275402,7.479323988684059,7.214194595882569,7.124416215856896
4,0,4,0.0,0.0,0.0,0.0,5.247975719996016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,16,36,9.152238189955547,9.119109923654763,9.1587037050376,9.15480998304841,9.193730969701132,9.187415060775526,9.198525173062688,9.195676509912568,9.167429993554173,9.151011662177845,9.10450510001819,9.154136573828351,9.129334841203853,9.076416363454179,9.11900675853024
37,16,37,10.334414121238066,10.290027457627165,10.36082966968769,10.34562607338292,10.3879402196444,10.364186713015524,10.37173229022977,10.391160642993912,10.341108710339775,10.360452771826655,10.331109215137378,10.378343102247376,10.333110812341713,10.239021119642294,10.208239284698921
38,16,38,7.539862626478917,7.530959369024364,7.556664394713148,7.607572224015891,7.647462104270601,7.538837183675365,7.4127712634173575,7.470931229122068,7.537387242069771,7.5246745395951296,7.731570563112639,7.78808434046907,7.616263839591061,7.061336764908699,7.183228576626887
39,16,39,8.86256190603391,8.816432630532052,8.862923138986726,8.837117496142183,8.853392400353194,8.81468208777418,8.77778592242638,8.75880310086754,8.75518379468266,8.784262506018898,8.763548331870648,8.769991731859768,8.751979045859487,8.694616539574334,8.632337247331266


In [1]:
# # 표준화

# # 표준화 데이터
# # 오브젝트 타입 컬럼 수치형으로 변환
# ntrain = xtrain_data
# object_columns = ['CARD_SIDO_NM', 'STD_CLSS_NM']
# ntrain[object_columns] = ntrain[object_columns].apply(lambda x: x.astype('category').cat.codes)

# # 예측 데이터 
# npredict = ntrain.drop(['CARD_SIDO_NM', 'STD_CLSS_NM', 201901], axis = 1)

# # 학습데이터
# nytrain = ntrain[202003]
# ntrain = ntrain.drop(['CARD_SIDO_NM', 'STD_CLSS_NM', 202003], axis = 1)

# # npredict
# for column in npredict.columns:
#     npredict[column] = (npredict[column] - np.mean(npredict[column])) / np.std(npredict[column])

# # ntrain
# for column in ntrain.columns:
#     ntrain[column] = (ntrain[column] - np.mean(ntrain[column])) / np.std(ntrain[column])

# # nytrain

# np_std = np.std(nytrain)
# np_mean = np.mean(nytrain)
# nytrain = (nytrain - np.mean(nytrain)) / np.std(nytrain)

# # 표준화

# # 2차원
# ntrain = np.array(ntrain)

# # 1차원
# nytrain = np.array(nytrain)

# # 2차원
# npredict = np.array(npredict)

# # 2차원 배열 -> 3차원 배열 변환
# # LSTM 3차원 Input shape 형식 [samples, timesteps, features]
# ntrain = ntrain.reshape(ntrain.shape[0], 1, ntrain.shape[1])
# npredict = npredict.reshape(npredict.shape[0], 1, npredict.shape[1])
# print(ntrain.shape)
# print(nytrain.shape)
# print(npredict.shape)

In [7]:
# 예측 데이터 
predict = xtrain.drop([201901], axis = 1)

# 학습데이터
ytrain = xtrain[202003]
xtrain = xtrain.drop([202003], axis = 1)

# 비교 타겟 (4월 데이터) 
Target = April[202004]

## Numpy 배열
- **딥러닝 학습을 위해 배열화 및 차원 변경**

In [8]:
# 2차원
xtrain = np.array(xtrain)

# 1차원
ytrain = np.array(ytrain)

# 2차원
predict = np.array(predict)

# 2차원 배열 -> 3차원 배열 변환
# LSTM 3차원 Input shape 형식 [samples, timesteps, features]
xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
predict = predict.reshape(predict.shape[0], 1, predict.shape[1])
print(xtrain.shape)
print(ytrain.shape)
print(predict.shape)

(697, 1, 16)
(697,)
(697, 1, 16)


# LSTM Deep learning

In [9]:
import keras
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dropout
print(tf.__version__)
print(keras.__version__)

2.3.0
2.4.3


### optimizer 설정
- **[OPTIMIZER](https://keras.io/ko/optimizers/#sgd)**

In [10]:
#OPTIMIZER = keras.optimizers.Adam(lr = 0.001, epsilon= None, decay = 0.0, amsgrad = True)
# OPTIMIZER = keras.optimizers.Adamax(lr = 0.1)
OPTIMIZER = keras.optimizers.Adagrad(lr = 0.1)
# OPTIMIZER = keras.optimizers.Nadam(lr = 0.1)
#OPTIMIZER = keras.optimizers.SGD(lr=0.001, nesterov=True)최고기록1.71
##OPTIMIZER = keras.optimizers.RMSprop(lr=0.01)도저히불가





### LSTM 특성

In [261]:
# model.add(layers.BatchNormalization()) 0과1사이 배치
# model.add(layers.Bidirectional(LSTM(200, activation = 'relu',return_sequences = True))) 양방향
# regularizers.l1_l2(l1 = 0.001, l2 = 0.001) 가중치 규제 설정법 
# model1.add(Dropout(0.2))


In [16]:
def build_model(input_shape=None):
    model1 = Sequential()
    model1.add(LSTM(200, activation = 'relu', return_sequences = True, input_shape = input_shape))
    model1.add(LSTM(200, activation = 'relu', return_sequences = True))
    model1.add(LSTM(200, activation = 'relu'))
    model1.add(Dense(256, activation='relu'))
    model1.add(Dense(128, activation='relu'))
    model1.add(Dense(64, activation='relu'))
    model1.add(Dense(32, activation='relu'))
    model1.add(Dense(1))
    return model1

In [375]:
# def build_model(input_shape=None):

#      model1 = Sequential()
#      model1.add(LSTM(256, activation = 'relu', return_sequences = True, input_shape = input_shape, recurrent_regularizer = regularizers.l1_l2(l1 = 0.001, l2 = 0.1)))
#      model1.add(LSTM(128, activation = 'relu', return_sequences = True))
#      model1.add(LSTM(64, activation = 'relu'))
#      model1.add(Dense(1))
#      return model1

In [376]:
# 모델
model1 = build_model((1,14))
model1.build(input_shape=(697,1,14))
model1.summary()

# 베스트 모델 저장
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='loss', save_weights_only=True, save_best_only=True)

# 모델 학습
model1.compile(optimizer = OPTIMIZER , loss = 'mse')
model1.fit(xtrain, ytrain, epochs= 105, batch_size=1, callbacks = [checkpoint])


Model: "sequential_100"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_320 (LSTM)              (None, 1, 256)            277504    
_________________________________________________________________
lstm_321 (LSTM)              (None, 1, 128)            197120    
_________________________________________________________________
lstm_322 (LSTM)              (None, 64)                49408     
_________________________________________________________________
dense_201 (Dense)            (None, 1)                 65        
Total params: 524,097
Trainable params: 524,097
Non-trainable params: 0
_________________________________________________________________
Epoch 1/105
697/697 [==============================] - 3s 5ms/step - loss: 2.4162
Epoch 2/105
697/697 [==============================] - 3s 4ms/step - loss: 1.0990
Epoch 3/105
697/697 [==============================] - 3s 4ms/step - loss: 1.005

In [377]:
#누적모델잘나왔을때 쓰시오.
#model1.save_weights('my_checkpoint.h5')

In [378]:
y_pred = model1.predict(predict, batch_size=1)
y_pred = y_pred.reshape(697,1)
y_pred = pd.DataFrame(y_pred)
y_pred = np.round(10**y_pred)-1
y_pred

,0
0,53532284.0
1,2556395520.0
2,799380992.0
3,6244701.0
4,0.0
...,...
692,1198258048.0
693,23083581440.0
694,10749511.0
695,414473248.0


In [368]:
# y_pred = abs(y_pred * np_std + np_mean)
# y_pred

# 예측데이터 비교

- **4월 데이터와 3월데이터의 증감비율 값**
```
> ytrain1 = xtrain_data[202003]
> error_val = Target - ytrain1
> len(error_val[error_val < 0]), len(error_val[error_val >= 0])
> (185, 512)
```


In [379]:
# 4월 이랑 RMSLE 비교 0에 가까울수록 좋음.
from sklearn.metrics import mean_squared_log_error as msle
print('4월 이랑 RMSLE 비교 0 에 가까울수록 좋음.')
print(np.sqrt(msle(Target, abs(y_pred))))

# 나의 예측 데이터와 3월 데이터 증감 비율 비교
ytrain1 = xtrain_data[202003]
error_val = y_pred[0] - ytrain1
print('-------------------------------------- ')
print('4월과 3월의 증감 비율 비슷할수록 좋음.')
print('(185, 512)')
print(len(error_val[error_val < 0]), len(error_val[error_val >= 0]))

# 결과물 저장
y_pred.to_csv('y_pred.csv',index = False)

4월 이랑 RMSLE 비교 0 에 가까울수록 좋음.
1.7977164518698787
-------------------------------------- 
4월과 3월의 증감 비율 비슷할수록 좋음.
(185, 512)
405 292


# 예측데이터 비교(Best Model)

In [19]:
best_model = build_model((1, 16))
best_model.build(input_shape=(697,1,16))
best_model.load_weights('best_weight.h5')
best_model.compile(optimizer = OPTIMIZER , loss = 'mse')

predictions = best_model.predict(predict)
predictions = np.round(10**predictions)-1
predictions = pd.DataFrame(predictions)
predictions

,0
0,84118304.0
1,2687591424.0
2,1044124032.0
3,9593602.0
4,0.0
...,...
692,1471721728.0
693,27405672448.0
694,18372680.0
695,485431168.0


In [372]:
# predictions = abs(predictions * np_std + np_mean) 

In [22]:
# 4월 이랑 베스트 모델 예측 RMSLE 비교 0에 가까울수록 좋음.
from sklearn.metrics import mean_squared_log_error as msle
print('4월 이랑 RMSLE 비교 0 에 가까울수록 좋음.')
print(np.sqrt(msle(Target, predictions)))

# 나의 예측 데이터와 3월 데이터 증감 비율 비교
ytrain1 = xtrain_data[202003]
error_val = predictions[0] - ytrain1
print('-------------------------------------- ')
print('4월과 3월의 증감 비율 비슷할수록 좋음.')
print('(185, 512)')
print(len(error_val[error_val < 0]), len(error_val[error_val >= 0]))

# 결과물 저장
predictions.to_csv('predictions.csv', index = False)

4월 이랑 RMSLE 비교 0 에 가까울수록 좋음.
1.6737514561520113
-------------------------------------- 
4월과 3월의 증감 비율 비슷할수록 좋음.
(185, 512)
424 273


In [25]:
def build_model():

    loss = keras.losses.MeanSquaredError()
    optimizer = keras.optimizers.Adagrad(lr=0.1)

    model = Sequential()
    model.add(LSTM(200, input_shape=(1,16), activation='relu', return_sequences=True))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(LSTM(200, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))

    model.build(input_shape=(697,1,16))
    model.load_weights('best_weight.h5')
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [48]:
April_amt = April_data[202004].to_frame()

In [56]:
April_xtrain = pd.concat([xtrain_data, April_amt], axis = 1)
April_xtrain = April_xtrain.set_index('CARD_SIDO_NM').reset_index()
April_xtrain_copy = April_xtrain.copy()

for column in April_xtrain_copy.columns[2:]:

    April_xtrain_copy[column] = np.log10(April_xtrain_copy[column] + 1)

April_xtrain_copy

obj = ['CARD_SIDO_NM','STD_CLSS_NM']
April_xtrain_copy[obj] = April_xtrain_copy[obj].apply(lambda x: x.astype('category').cat.codes)

ytrain = April_xtrain_copy[202004]
predict = April_xtrain_copy.drop([201901, 201902], axis = 1)
xtrain = April_xtrain_copy.drop([201901, 202004], axis = 1)

# 2차원
xtrain = np.array(xtrain)

# 1차원
ytrain = np.array(ytrain)

# 2차원
predict = np.array(predict)

# 2차원 배열 -> 3차원 배열 변환
# LSTM 3차원 Input shape 형식 [samples, timesteps, features]
xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
predict = predict.reshape(predict.shape[0], 1, predict.shape[1])
print(xtrain.shape)
print(ytrain.shape)
print(predict.shape)




(697, 1, 16)
(697,)
(697, 1, 16)


In [61]:
## 5월 AMT 생성
best_model = build_model()
may_amt = best_model.predict(predict)
may_amt = pd.DataFrame(may_amt, columns=[202005])
may_amt

,202005
0,7.937012195587158
1,9.683632850646973
2,9.060786247253418
3,6.99231481552124
4,0.20131295919418335
...,...
692,9.19467544555664
693,10.481791496276855
694,7.270424842834473
695,8.685564994812012


In [65]:
May_xtrain = pd.concat([April_xtrain_copy, may_amt], axis = 1)
May_xtrain_copy = May_xtrain.copy()


ytrain = May_xtrain_copy[202005]
predict = May_xtrain_copy.drop([201901, 201902, 201903], axis = 1)
xtrain = May_xtrain_copy.drop([201901, 201902, 202005], axis = 1)

# 2차원
xtrain = np.array(xtrain)

# 1차원
ytrain = np.array(ytrain)

# 2차원
predict = np.array(predict)

# 2차원 배열 -> 3차원 배열 변환
# LSTM 3차원 Input shape 형식 [samples, timesteps, features]
xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
predict = predict.reshape(predict.shape[0], 1, predict.shape[1])
print(xtrain.shape)
print(ytrain.shape)
print(predict.shape)

(697, 1, 16)
(697,)
(697, 1, 16)


In [68]:
## 6월 AMT 생성
best_model = build_model()
june_amt = best_model.predict(predict)
june_amt = pd.DataFrame(june_amt, columns=[202006])
june_amt

,202006
0,7.933596134185791
1,9.7522554397583
2,9.075196266174316
3,6.9087605476379395
4,0.18728461861610413
...,...
692,9.23343563079834
693,10.60207462310791
694,7.2779669761657715
695,8.694969177246094


In [69]:
June_xtrain = pd.concat([May_xtrain_copy, june_amt], axis = 1)
June_xtrain_copy = June_xtrain.copy()


ytrain = June_xtrain_copy[202006]
predict = June_xtrain_copy.drop([201901, 201902, 201903, 201904], axis = 1)
xtrain = June_xtrain_copy.drop([201901, 201902, 201903, 202006], axis = 1)

# 2차원
xtrain = np.array(xtrain)

# 1차원
ytrain = np.array(ytrain)

# 2차원
predict = np.array(predict)

# 2차원 배열 -> 3차원 배열 변환
# LSTM 3차원 Input shape 형식 [samples, timesteps, features]
xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
predict = predict.reshape(predict.shape[0], 1, predict.shape[1])
print(xtrain.shape)
print(ytrain.shape)
print(predict.shape)

(697, 1, 16)
(697,)
(697, 1, 16)


In [76]:
## 7월 AMT 생성
best_model = build_model()
July_amt = best_model.predict(predict)
July_amt = np.round(10**July_amt) - 1
July_amt = pd.DataFrame(July_amt, columns=['AMT'])
July_amt

,AMT
0,85331512.0
1,7308394496.0
2,1227115136.0
3,7095322.0
4,0.0
...,...
692,1871735424.0
693,49683238912.0
694,17737688.0
695,491193600.0


In [87]:
April_amt.columns = ['AMT']
sub_amt = pd.concat([April_amt, July_amt])
sub_amt = sub_amt.set_index('AMT').reset_index()
sub_amt

,AMT
0,88823988.0
1,4708346820.0
2,1121028924.0
3,14360780.0
4,0.0
...,...
1389,1871735424.0
1390,49683238912.0
1391,17737688.0
1392,491193600.0


In [82]:
sub = pd.read_csv('submission.csv')

In [84]:
sub.drop('AMT', axis = 1, inplace = True)

In [88]:
finish = pd.concat([sub, sub_amt], axis = 1)
finish

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,88823988.0
1,1,202004,강원,골프장 운영업,4708346820.0
2,2,202004,강원,과실 및 채소 소매업,1121028924.0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,14360780.0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.0
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1871735424.0
1390,1390,202007,충북,한식 음식점업,49683238912.0
1391,1391,202007,충북,호텔업,17737688.0
1392,1392,202007,충북,화장품 및 방향제 소매업,491193600.0


In [90]:
finish.to_csv('finish.csv', index = False , encoding = 'utf-8-sig')

In [91]:
pd.read_csv('finish.csv')

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,88823988.0
1,1,202004,강원,골프장 운영업,4708346820.0
2,2,202004,강원,과실 및 채소 소매업,1121028924.0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,14360780.0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.0
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1871735424.0
1390,1390,202007,충북,한식 음식점업,49683238912.0
1391,1391,202007,충북,호텔업,17737688.0
1392,1392,202007,충북,화장품 및 방향제 소매업,491193600.0


In [92]:
July_amt[:10]

,AMT
0,85331512.0
1,7308394496.0
2,1227115136.0
3,7095322.0
4,0.0
5,9481.0
6,302020704.0
7,9147346944.0
8,8.0
9,835485440.0


# 보완할점 및 새로운 시도
- **가중치가 계속 변하기 떄문에 현재 방식으로는 정확한 예측이 어렵다.**
- **best model을 사용하여 7월을 예측하였기 때문에 데이터가 증가항목은 계속증가하고 감소항목은 계속 감소하는 오류가 발생하였다.**
- **ARIMA 모델을 사용하여 step을 설정하여서 7월을 예측해보면 더 나은 결과를 얻을 수 있지 않을까 예상을 해본다.**